In [1]:
import pandas as pd


significant_snp = pd.read_excel('/home/esp/data_analyze/01.06.2025_v2/data/init/alleli_alz.xlsx')
all_snp = pd.read_csv('/home/esp/data_analyze/01.06.2025_v2/data/output/gwas_results.csv', sep=';')
xxx = pd.read_excel('/home/esp/data_analyze/01.06.2025_v2/data/init/almagul_r.xlsx')
len(xxx)

13

In [2]:
# Фильтрация по SNP
filtered_snp_xxx = all_snp[all_snp['SNP'].isin(xxx['SNP'])]

# Вывод результата
print(f"Количество совпадающих SNP: {len(filtered_snp_xxx)}")

Количество совпадающих SNP: 13


In [3]:
filtered_snp_xxx

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
66622,3,rs11708767,151119726,A,0.3023,0.3379,G,0.7939,0.372900,0.8490
79793,4,rs11723502,185804320,C,0.1473,0.1172,T,1.0800,0.298700,1.3010
126343,7,rs7805803,50225391,A,0.3333,0.4034,G,2.8780,0.089790,0.7393
235977,19,rs184017,45394969,C,0.3450,0.2310,A,8.7020,0.003178,1.7530
235979,19,rs2075650,45395619,G,0.2326,0.1276,A,10.3300,0.001311,2.0720
235981,19,rs157581,45395714,G,0.3488,0.2414,A,7.6260,0.005752,1.6840
235982,19,rs34404554,45395909,G,0.2326,0.1276,C,10.3300,0.001311,2.0720
235983,19,rs157582,45396219,A,0.3450,0.2207,G,10.4800,0.001208,1.8600
235984,19,rs59007384,45396665,T,0.3140,0.1931,G,10.6300,0.001110,1.9120
235991,19,rs8106922,45401666,G,0.2287,0.3448,A,8.9420,0.002786,0.5633


In [4]:
# Фильтрация по SNP
filtered_snp = all_snp[all_snp['SNP'].isin(significant_snp['SNP'])]

# Вывод результата
print(f"Количество совпадающих SNP: {len(filtered_snp)}")

Количество совпадающих SNP: 10


In [5]:
filtered_snp

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR
24117,1,rs6656401,207692049,A,0.14730,0.14830,G,0.001060,0.97400,0.9922
24155,1,rs3818361,207784968,T,0.25580,0.25860,C,0.005628,0.94020,0.9854
131859,7,rs11771145,143110762,A,0.41470,0.45520,G,0.908100,0.34060,0.8482
137462,8,rs2279590,27456253,A,0.34880,0.37240,G,0.328900,0.56630,0.9028
137464,8,rs11136000,27464519,T,0.34110,0.36900,C,0.463000,0.49620,0.8853
137465,8,rs9331888,27468862,C,0.39920,0.34140,G,1.963000,0.16120,1.2820
175736,11,rs3851179,85868640,A,0.44190,0.40000,G,0.982300,0.32160,1.1880
235999,19,rs405509,45408836,C,0.33330,0.43450,A,5.889000,0.01523,0.6508
236009,19,rs429358,45411941,0,0.00000,0.00000,T,NaN,NaN,NaN
236011,19,rs7412,45412079,T,0.04297,0.03448,C,0.264800,0.60690,1.2570


In [6]:
from Bio import Entrez
import pandas as pd
import time

Entrez.email = "maxat.suieubayev@nu.edu.kz"
Entrez.api_key = "ae25f7cac7ed6f393f8ad904176767c56808"

def annotate_snp(rsid):
    try:
        handle = Entrez.esearch(db="snp", term=rsid)
        record = Entrez.read(handle)
        if not record["IdList"]:
            return {"rsID": rsid}

        snp_id = record["IdList"][0]
        summary = Entrez.esummary(db="snp", id=snp_id)
        summary_data = Entrez.read(summary)
        docsum = summary_data["DocumentSummarySet"]["DocumentSummary"][0]

        return {
            "rsID": rsid,
            "Chr": docsum.get("CHR", "—"),
            "Position": docsum.get("CHRPOS", "—"),
            "Gene": docsum["GENES"][0]["NAME"] if docsum.get("GENES") else "—",
            "SNP Class": docsum.get("SNP_CLASS", "—"),
            "Global MAF": docsum.get("GLOBAL_MAF", "—"),
            "Alleles": docsum.get("ALLELE", "—"),
            "Function Class": docsum.get("FXN_CLASS", "—")
        }
    except Exception as e:
        print(f"⚠️ Ошибка аннотации {rsid}: {e}")
        return {"rsID": rsid}

rs_ids = filtered_snp['SNP']
rs_ids = rs_ids[rs_ids.str.startswith('rs')].unique()

# Собираем результаты
snp_summaries = []
for rs in rs_ids:
    print(f"🔍 Обработка {rs}...")
    summary = annotate_snp(rs)
    snp_summaries.append(summary)
    time.sleep(0.4)

# Создание таблицы
summary_df = pd.DataFrame(snp_summaries)
summary_df.to_csv('/home/esp/data_analyze/01.06.2025_v2/data/output/snp_annotation.csv', index=False)


🔍 Обработка rs6656401...
🔍 Обработка rs3818361...
🔍 Обработка rs11771145...
🔍 Обработка rs2279590...
🔍 Обработка rs11136000...
🔍 Обработка rs9331888...
🔍 Обработка rs3851179...
🔍 Обработка rs405509...
🔍 Обработка rs429358...
🔍 Обработка rs7412...


In [7]:
import pandas as pd

# Загрузка результатов
path = '/home/esp/data_analyze/01.06.2025_v2/data/output/first_step/gwas_results.PHENO.glm.logistic.hybrid'
gwas_df = pd.read_csv(path, delim_whitespace=True)

# Посмотрим на структуру
print(gwas_df.columns)
print(gwas_df.head())

# Отфильтруем значимые SNP
significant_snps = gwas_df[gwas_df['P'] < 0.05]
print(f"🧬 Найдено значимых SNP: {len(significant_snps)}")

# Сохраним
significant_snps.to_csv('significant_snps.csv', index=False)


/tmp/ipykernel_5361/3393079589.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  gwas_df = pd.read_csv(path, delim_whitespace=True)


Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'PROVISIONAL_REF?', 'A1',
       'OMITTED', 'A1_FREQ', 'FIRTH?', 'TEST', 'OBS_CT', 'OR', 'LOG(OR)_SE',
       'Z_STAT', 'P', 'ERRCODE'],
      dtype='object')
  #CHROM  POS               ID REF ALT PROVISIONAL_REF? A1 OMITTED  A1_FREQ  \
0      0    0      10_24948653   G   .                Y  .       G      0.0   
1      0    0      11_14272702   C   .                Y  .       C      0.0   
2      0    0         1A-Cyt-2   A   .                Y  .       A      0.0   
3      0    0  1kg_11_34998496   G   .                Y  .       G      0.0   
4      0    0  1kg_11_35000864   C   .                Y  .       C      0.0   

  FIRTH? TEST  OBS_CT  OR  LOG(OR)_SE  Z_STAT   P               ERRCODE  
0      N  ADD     282 NaN         NaN     NaN NaN  CONST_OMITTED_ALLELE  
1      N  ADD     279 NaN         NaN     NaN NaN  CONST_OMITTED_ALLELE  
2      N  ADD     283 NaN         NaN     NaN NaN  CONST_OMITTED_ALLELE  
3      N  ADD     278 NaN 

/tmp/ipykernel_5361/3393079589.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gwas_df = pd.read_csv(path, delim_whitespace=True)


In [8]:
significant_snps.head()

,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
43,0,0,1kg_1_159759291,G,C,Y,C,G,0.067857,N,ADD,280,2.307690,0.365499,2.28796,0.022140,.
66,0,0,1kg_1_159801856,A,G,Y,G,A,0.072202,N,ADD,277,2.024920,0.351717,2.00596,0.044860,.
78,0,0,1kg_1_159817869,C,T,Y,T,C,0.064516,N,ADD,279,2.103950,0.370116,2.00968,0.044465,.
88,0,0,1kg_1_159826686,G,A,Y,A,G,0.071930,N,ADD,285,0.493630,0.353368,-1.99783,0.045735,.
114,0,0,1kg_1_241025971,C,A,Y,A,C,0.152330,N,ADD,279,0.574591,0.247892,-2.23524,0.025402,.


In [9]:
# Фильтруем SNP, повышающие риск
risk_snps = significant_snps[(significant_snps['P'] < 0.05) & (significant_snps['LOG(OR)_SE'] < 0.165)]

# Смотрим результат
print(f"🧬 SNP, повышающие риск: {len(risk_snps)}")
display(risk_snps[['ID', 'OR', 'P', 'Z_STAT']].sort_values(by='P'))
#risk_snps.to_csv('risk_snps_OR>8.csv')
rs_ids = risk_snps['ID']
#rs_ids = rs_ids[rs_ids.str.startswith('rs')].unique()
print('RS counts: ', len(rs_ids))
rs_ids.head()

🧬 SNP, повышающие риск: 207


,ID,OR,P,Z_STAT
149401,imm_9_34914487,1.818590,0.000288,3.62559
251443,rs1545469,0.564016,0.000382,-3.55211
189865,rs2374710,1.660190,0.002075,3.07925
149297,imm_9_34847969,1.655210,0.002251,3.05496
149409,imm_9_34918487,1.627620,0.002433,3.03161
...,...,...,...,...
231420,rs2364835,1.371790,0.049204,1.96682
247087,rs5757244,0.724641,0.049209,-1.96678
217295,rs8060063,0.735468,0.049222,-1.96667
131451,rs10954351,1.379780,0.049467,1.96455


RS counts:  207


4756          rs10489163
4789           rs4295917
8418            rs284176
19410    imm_1_181666236
19414    imm_1_181667838
Name: ID, dtype: object

In [10]:
rs_ids = risk_snps['ID']
rs_ids = rs_ids[rs_ids.str.startswith('rs')].unique()

# Собираем результаты
snp_summaries = []
for rs in rs_ids:
    print(f"🔍 Обработка {rs}...")
    summary = annotate_snp(rs)
    snp_summaries.append(summary)
    time.sleep(0.4)

# Создание таблицы
summary_df = pd.DataFrame(snp_summaries)
#summary_df.to_csv('/home/esp/data_analyze/01.06.2025_v2/data/output/or>8_snp_annotation.csv', index=False)
len(summary_df)

🔍 Обработка rs10489163...
🔍 Обработка rs4295917...
🔍 Обработка rs284176...
🔍 Обработка rs1019969...
🔍 Обработка rs10754596...
🔍 Обработка rs2073316...
🔍 Обработка rs2365772...
🔍 Обработка rs4233921...
🔍 Обработка rs7419716...
🔍 Обработка rs1432311...
🔍 Обработка rs934390...
🔍 Обработка rs13006885...
🔍 Обработка rs2042555...
🔍 Обработка rs9973944...
🔍 Обработка rs6550578...
🔍 Обработка rs2638148...
🔍 Обработка rs9819616...
🔍 Обработка rs6777787...
🔍 Обработка rs783511...
🔍 Обработка rs960212...
🔍 Обработка rs3773989...
🔍 Обработка rs10022491...
🔍 Обработка rs261119...
🔍 Обработка rs852564...
🔍 Обработка rs2199787...
🔍 Обработка rs255332...
🔍 Обработка rs28129...
🔍 Обработка rs11952081...
🔍 Обработка rs2962783...
🔍 Обработка rs13362387...
🔍 Обработка rs11741906...
🔍 Обработка rs2517523...
🔍 Обработка rs2905759...
🔍 Обработка rs2523843...
🔍 Обработка rs9263473...
🔍 Обработка rs3131000...
🔍 Обработка rs3095310...
🔍 Обработка rs3095302...
🔍 Обработка rs3095301...
🔍 Обработка rs3131003...
🔍 

114

In [11]:
len(rs_ids)

114

In [12]:
print(f"Количество совпадающих SNP: {len(significant_snp)}")

Количество совпадающих SNP: 36


In [13]:
# Фильтрация по SNP
filtered_snp_2 = gwas_df[gwas_df['ID'].isin(significant_snp['SNP'])]

# Вывод результата
print(f"Количество совпадающих SNP: {len(filtered_snp_2)}")
filtered_snp_2


Количество совпадающих SNP: 10


,#CHROM,POS,ID,REF,ALT,PROVISIONAL_REF?,A1,OMITTED,A1_FREQ,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,Z_STAT,P,ERRCODE
24117,1,207692049,rs6656401,G,A,Y,A,G,0.147163,N,ADD,282,0.998436,0.234926,-0.006664,0.994683,.
24155,1,207784968,rs3818361,C,T,Y,T,C,0.256272,N,ADD,279,0.955811,0.196751,-0.229709,0.818318,.
131859,7,143110762,rs11771145,G,A,Y,A,G,0.432624,N,ADD,282,0.846658,0.171711,-0.969410,0.332341,.
137462,8,27456253,rs2279590,G,A,Y,A,G,0.365108,N,ADD,278,0.942221,0.167688,-0.354917,0.722652,.
137464,8,27464519,rs11136000,C,T,Y,T,C,0.361702,N,ADD,282,0.943255,0.166608,-0.350635,0.725862,.
137465,8,27468862,rs9331888,G,C,Y,C,G,0.366426,N,ADD,277,1.231340,0.169768,1.225820,0.220266,.
175736,11,85868640,rs3851179,G,A,Y,A,G,0.424460,N,ADD,278,1.232320,0.174026,1.200380,0.229993,.
235999,19,45408836,rs405509,A,C,Y,C,A,0.393617,N,ADD,282,0.692693,0.176517,-2.080070,0.037519,.
236009,19,45411941,rs429358,T,.,Y,.,T,0.000000,N,ADD,274,NaN,NaN,NaN,NaN,CONST_OMITTED_ALLELE
236011,19,45412079,rs7412,C,T,Y,T,C,0.040493,N,ADD,284,1.414400,0.438487,0.790686,0.429127,.
